In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as WN
from pyspark.sql import types as TP

In [0]:
spark = SparkSession.builder.appName("snow").master("local[2]").getOrCreate()

In [0]:
filepath = "E:/workforce/spark/data/Bengaluru_House_Data.csv"
output_path = "E:/workforce/spark/data/output/"

In [0]:
def read_file(spark,filepath):
    return spark.read.format("csv").options(path=datafile,header=True,inferSchema = True,delimiter = ",").load()

def get_db_credentials():
    db_cred = {
    "url" :"jdbc:postgresql://localhost:5432/db101",
    "driver": "org.postgresql.Driver",
    "user": "postgres",
    "password": "khan102030"
    }
    return db_cred
    

In [0]:
dataframe = read_file(spark,filepath)

In [0]:
# dataframe.show()
# dataframe.count()
# dataframe.rdd.getNumPartitions()
dataframe.groupBy("area_type").agg(FS.count("*").alias("count")).show()

In [0]:
# Write to file and database and see result: patrition: 1

### Case-1: Partition-1

In [0]:
# It create one file and partition is one
dataframe.write.format("csv").mode('overwrite').option("path",output_path).save()

In [0]:
# It creates on table
dataframe.select("*").write.format("jdbc").options(**db_cred).\
option("dbtable", 'house102').mode("overwrite").save()

### Case-2: Partition-3

In [0]:
dataframe2 = dataframe.repartition(3)

In [0]:
dataframe2.rdd.getNumPartitions()

In [0]:
# Write to file and database and see result: patrition: 3
# It create three file and partition is 3 and each executers write
dataframe2.write.format("csv").mode('overwrite').option("path",output_path).save()

In [0]:
# # It creates a table
dataframe2.select("*").write.format("jdbc").options(**db_cred).\
option("dbtable", 'house102').mode("overwrite").save()

### Case-3 Partition-3 and PartitionBy

In [0]:
# It creats folder by column and creates files as per partition, in this case 3 files
dataframe2.write.format("csv").mode('overwrite').option("path",output_path).partitionBy("area_type").save()

In [0]:
# # It creates a table
dataframe2.select("*").write.format("jdbc").options(**db_cred).\
option("dbtable", 'house102').mode("overwrite").partitionBy("area_type").save()

### Case-4 Partition-3 and bucketBy
Bucketing is an optimization method that breaks down data into more manageable parts (buckets) to determine the data partitioning while it is written out. The motivation for this method is to make successive reads of the data more performant for downstream jobs if the SQL operators can make use of this property.

here bucketBy distributes data to a fixed number of buckets (16 in our case) and can be used when the number of unique values is not limited. If the number of unique values is limited, it's better to use a partitioning instead of a bucketing.
## Note
Bucketing (CLUSTERED BY) is not supported for **Delta tables** because Delta Lake utilizes superior optimization techniques like **Z-Ordering and Liquid Clustering** for data skipping. 

In [0]:
#1: Save does not support bucketBy right now.
#2: It creats
dataframe2.write.mode('overwrite').option("path",output_path).bucketBy(16,"area_type").\
saveAsTable('bucketed', format='parquet')

In [0]:
# # It creates a table
dataframe2.select("*").write.format("jdbc").options(**db_cred).\
option("dbtable", 'bucketed').mode("overwrite").bucketBy(16,"area_type").saveAsTable("bucketed101")